In [ ]:
from tensorflow import keras
from sklearn.model_selection import train_test_split

(train_input, train_target), (test_input, test_target) =\
    keras.datasets.fashion_mnist.load_data()

In [ ]:
# Conv2D (합성곱 층)을 사용하기 위해 reshape 수행
train_scaled = train_input.reshape(-1,28,28,1) / 255.0
train_scaled, val_scaled, train_target, val_target = train_test_split (
    train_scaled, train_target, test_size = 0.2, random_state = 42
)


In [ ]:
model = keras.Sequential()

# (3,3) 사이즈의 필터 32개를 가지는 합성곱 층, 첫 번째 층이므로 input_shape를 정의하며 activation으로 relu함수를 사용하고 same padding 사용
model.add(keras.layers.Conv2D(32, kernel_size=3, activation='relu', padding='same', input_shape = (28,28,1)))

# (2,2) Max Pooling 층 (28, 28, 1) -> (14, 14, 32)
model.add(keras.layers.MaxPooling2D(2))

# (3,3) 사이즈의 필터 64개를 가지는 합성곱 층, 이전 Max Pooling 층에서 (2,2) 풀링을 수행했으므로 입력은 (14, 14, 32)
model.add(keras.layers.Conv2D(64, kernel_size=3, activation='relu', padding='same'))

# (2,2) Max Pooling 층 (14, 14, 64) -> (7, 7, 64)
model.add(keras.layers.MaxPooling2D(2))

# 1차원으로 펼치기 위한 Flatten 층
model.add(keras.layers.Flatten())

# 100개의 뉴런을 가지는 은닉층
model.add(keras.layers.Dense(100, activation='relu'))

# 과대적합을 줄이기 위한 Dropout 층
model.add(keras.layers.Dropout(0.4))

# softmax 출력층
model.add(keras.layers.Dense(10, activation='softmax'))

model.summary()

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

# callbacks
checkpoint_cb = keras.callbacks.ModelCheckpoint('best-cnn-model.h5', save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)
history = model.fit(train_scaled, train_target, epochs = 20,
                    validation_data = (val_scaled, val_target), 
                    callbacks = [checkpoint_cb, early_stopping_cb])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
model.evaluate(val_scaled, val_target)

In [ ]:
# plt.imshow(val_scaled[0].reshape(28,28), cmap='gray_r')
preds = model.predict(val_scaled[0:1])
plt.bar(range(1, 11), preds[0])
plt.show()